In [1]:
import os
import sys
from typing import Optional, Any

from dotenv import load_dotenv, find_dotenv
import numpy as np

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

/media/D/miniconda/envs/llm_intro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# make sure GOOGLE_API_KEY (or any other llm provider, must exactly the same)
_ = load_dotenv(find_dotenv())
# or put it into variable and pass it to llm model 
# google_api_key = os.getenv('GOOGLE_API_KEY') # pass this to the model creation

# Agent Concept

![](../media/agent_tools.png)

What is an AI Agent? 
An AI agent is a program that uses a language model to understand a task and decide the best way to complete it using available tools autonomously.
A key idea in AI agents is the set of **Tools** (functions) they can use to get the job done. 
For each request, the AI plans the best approach, deciding what steps and tools are needed and in what order to complete the task.

For this example, we will create an Agent that could get the weather from external API, either by specify the location/city or by trying to use current location.

In [35]:
"""
agent that we use as example:
    1. get weather
    2. get location (dummy, returning random between Jakarta, Seoul, Tokyo)
"""
from datetime import datetime
import zoneinfo as zi
import requests
from langchain_core.tools import BaseTool
from langchain_core.tools import Tool, tool, StructuredTool
from pydantic import BaseModel, Field
from typing import Type, Optional
import random

class Location(BaseModel):
    location: str = Field(description='Location to find weather')
    
@tool
def get_weather(location: Optional[str]='Jakarta') -> str:
    """Get the weather via API in a location (provided, Jakarta as default).
    A tool designed to gather accurate information on weather around the globe. 
    Useful for when you need to answer questions about the weather. 
    Input should be the location.
    """
    WEATHERAPI_API_KEY='8a6c94f591c4401c9e8152836251301'
    res = requests.get(f'https://api.weatherapi.com/v1/current.json?key=8a6c94f591c4401c9e8152836251301&q={location}&aqi=no')
    return "For {} temperature: {:.2F} and weather: {}".format(
        location,
        res.json()['current']['temp_c'],
        res.json()['current']['condition']['text']
        )
@tool
def get_current_location() -> str:
    """
    Get the current location of a user (approxmate).
    Input: None
    Output: City name 
    """

    random_city = ['Jakarta', 'Seoul', 'Tokyo', 'Singapore', 'Kuala Lumpur']
    return random.choice(random_city)

The function is a tool already (after using decorator). In order to test the tool, call `.run()` method.

In [21]:
get_weather.run('Jakarta')

'For Jakarta temperature: 28.30 and weather: Partly cloudy'

In [36]:
get_current_location.run({})

'Seoul'

Use langgraph function to create a react agent (stands for Reasoning and Action agent). On a high level, given a query, the model will [be](https://www.leewayhertz.com/react-agents-vs-function-calling-agents/):
1. Break down the task into smaller step, choosing the required tools
2. Exceute the step based on the tools
3. Observe the result (should it use another tools or return the answer)
4. Give response


In [44]:
from langgraph.prebuilt import create_react_agent

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# query = "How's the weather in Jakarta?"
# query = "What's my current location, and what's the temperature?"
query = "What's the temperature on my current location?" # the agent is smart enough to call the get_location tools first, before pass it to the get_weather tools
agent_executor = create_react_agent(model=llm, tools=[get_weather, get_current_location])
message = agent_executor.invoke({
    'messages': [
        ('human', query)
    ]
})
message['messages'][-1].content

"The temperature in Singapore is 25.40 degrees Celsius and it's partly cloudy.\n"

## Saving Checkpoint

Langgraph provides a way to save a chat session by having thread_id. 

In [9]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

agent_executor = create_react_agent(model=llm, tools=[get_weather, get_weather_average], checkpointer=memory)

# create configuration for Thread Id
config = {"configurable": {"thread_id": "test-thread"}}
message = agent_executor.invoke({
    'messages': [
        ('human', query)
    ]},
    config
)

In [10]:
message['messages'][-1]

AIMessage(content='The temperature in Seoul is -9.8°C and the weather is clear.  In Tokyo it is 3.3°C and partly cloudy. Jakarta is 28.1°C and partly cloudy, while Kuala Lumpur is 26.2°C and partly cloudy.\n\nThe average temperature across all four cities is 11.95°C.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-2e08119f-4285-4ff7-8525-ea71a12fdde5-0', usage_metadata={'input_tokens': 234, 'output_tokens': 79, 'total_tokens': 313, 'input_token_details': {'cache_read': 0}})

In [11]:
m = agent_executor.invoke({
    'messages': [
        ('human', 'What is the city that I ask previously?')
    ]},
    config
)

In [12]:
m['messages'][-1]

AIMessage(content='The cities you asked about previously were Seoul, Tokyo, Jakarta, and Kuala Lumpur.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a2f664c6-b669-45b4-8144-3b1e43cab23d-0', usage_metadata={'input_tokens': 322, 'output_tokens': 18, 'total_tokens': 340, 'input_token_details': {'cache_read': 0}})